In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE97475"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE97475"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE97475.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE97475.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE97475.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this appears to be a gene expression study of healthy hepatitis B vaccine recipients.
# The summary mentions "transcriptomic" data collection, which suggests gene expression data is available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Height (trait)
# Looking at 'subjects.demographics.height: NA' - all values are NA, so height data is not available
trait_row = None  # Setting to None since all values appear to be NA

# Age Data
# Key 81 contains age information with multiple unique values
age_row = 81

# Gender Data
# Key 118 contains gender information with two values: Male and Female
gender_row = 118

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert height data to a float value in cm."""
    if not value or value == 'NA':
        return None
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Try to convert to float, assuming height is in cm
    try:
        return float(value)
    except ValueError:
        return None

def convert_age(value):
    """Convert age data to integer."""
    if not value or value == 'NA':
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary (0 for Female, 1 for Male)."""
    if not value or value == 'NA':
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    return None

# 3. Save Metadata
# Determine trait availability based on whether trait_row is None
is_trait_available = trait_row is not None

# Initial filtering on usability and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None and the clinical data file exists
import os

if trait_row is not None:
    clinical_data_path = f"{in_cohort_dir}/clinical_data.csv"
    if os.path.exists(clinical_data_path):
        # Load the clinical data
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features using the library function
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the output
        preview = preview_df(selected_clinical)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the selected clinical features
        selected_clinical.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical features saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")
        print("Skipping clinical feature extraction.")
else:
    print(f"Trait data ({trait}) is not available in this dataset.")
    print("Skipping clinical feature extraction.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Let's analyze the gene identifiers
from typing import List

def is_human_gene_symbol(gene_ids: List[str]) -> bool:
    """
    Determine if a list of identifiers are human gene symbols.
    
    Human gene symbols are typically:
    - Short (usually 1-8 characters)
    - Uppercase letters
    - May contain numbers, usually at the end
    - Sometimes include hyphens
    
    Returns True if identifiers appear to be human gene symbols.
    """
    # Check a sample of genes that are clearly human gene symbols
    known_human_genes = {'A1BG', 'AAAS', 'AACS', 'AAMP', 'AARS'}
    
    # Count how many match our known set
    matches = sum(1 for gene in gene_ids if gene in known_human_genes)
    
    # If we have multiple matches to known gene symbols, these are likely human gene symbols
    if matches >= 3:
        return True
        
    return False

# Check the sample gene IDs shown in the output
sample_genes = ['A1BG', 'A26C3', 'A2LD1', 'A4GNT', 'AAAS', 'AACS', 'AACSL', 'AADACL1',
                'AADAT', 'AAGAB', 'AAK1', 'AAMP', 'AARS', 'AARS2', 'AARSD1', 'AASDH',
                'AASDHPPT', 'AASS', 'AATF', 'AATK']

requires_gene_mapping = not is_human_gene_symbol(sample_genes)
print(f"requires_gene_mapping = {requires_gene_mapping}")


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Let's load the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Normalize gene symbols in the gene expression data
# The gene_data object from Step 3 already contains our gene expression data
# Let's first load it again to make sure we have the correct data
gene_data = get_genetic_data(matrix_file)

# Normalize gene symbols using the NCBI Gene database information
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Attempt to link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Therefore, we need to indicate that trait data is not available

# Create a minimal clinical dataframe with the trait column (containing NaNs)
sample_ids = gene_data.columns
# Create a DataFrame with sample IDs and a placeholder Height column
minimal_clinical_df = pd.DataFrame(index=sample_ids)
minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values
minimal_clinical_df.index.name = 'Sample'

# Save this minimal clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Minimal clinical data with placeholder {trait} column saved to {out_clinical_data_file}")

# Create a linked dataset 
linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
linked_data.index.name = 'Sample'
print(f"Linked data shape: {linked_data.shape}")

# 5. Validate and save cohort information
# trait_row was None in Step 2, so we know Height data is not available
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,  # Explicitly set to False instead of None
    df=linked_data,
    note=note
)

# Since there is no trait data, the dataset is not usable for our association study
print("Dataset does not contain Height data and cannot be used for association studies.")


### Step 6: Data Normalization and Linking

In [ ]:
import numpy as np
import os

# 1. Let's load the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
gene_data = get_genetic_data(matrix_file)

# Normalize gene symbols using the NCBI Gene database information
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
sample_ids = normalized_gene_data.columns
minimal_clinical_df = pd.DataFrame(index=sample_ids)
minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

# If we have age and gender data from Step 2, add those columns
if age_row is not None:
    minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

if gender_row is not None:
    minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

minimal_clinical_df.index.name = 'Sample'

# Save this minimal clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Create a linked dataset 
linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
linked_data.index.name = 'Sample'
print(f"Linked data shape: {linked_data.shape}")

# We would normally handle missing values here, but since all trait values are missing,
# the dataset will be empty after removing samples with missing trait values
# Therefore, we'll skip that step

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")